In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from scipy import ndimage
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

### Real(ish) World Example
#### Creating a active transit priority index for Oakland

Please borrow and improve on the code! <br>
It is set up to run with minimal input from the user. <br>
It uses census shapefiles, crash data from TIMS, and census demographic information.

### 1. Prepare Data

#### 1.1 Set Geography for Analysis

Start by selecting the spatial information for the Census block group. <br>
Start by Downloading Geography <br>
Files are from Census

In [ ]:
census_places = gpd.read_file("Spatial Files/tl_2019_06_place.shp")
block_gr = gpd.read_file("Spatial Files/tl_2019_06_bg.shp")

In [ ]:
# Start by selecting Oakland from places file using name
place = census_places[census_places['NAME'] == 'Oakland']

In [ ]:
# Use overlay function to select only for Census Block groups inside of Oakland
bl_gr = gpd.overlay(place, block_gr, how = "intersection")

In [ ]:
# GeoID_2 represents the block group code
bl_gr.head(1)

In [ ]:
# Make a simple copy of the bl_stock that retains id and geography
bl_simp = bl_gr[['GEOID_2','geometry']].reset_index()
place_simp = place[['NAME','geometry']]

In [ ]:
# See what we have
bl_simp.plot(figsize = (5, 5), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.5).set_axis_off()

In [ ]:
# Let's drop the ugly hanging Tract that is entirely in the bay.
# I identified it by fiding the tract with the most water in it
bl_simp = bl_simp[bl_simp['GEOID_2'] != '060019900000']

In [ ]:
bl_simp.plot(figsize = (5, 5), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.5).set_axis_off()

#### 1.2 Attach Geospatial Data - Ped and Bike Crashes
This part of the code loads and cleans crash data from TIMS. <br>
The crashes are for 5 years of data for Oakland.

In [ ]:
ped_cr = gpd.read_file("Spatial Files/oakland_ped_collisions.shp")
bike_cr = gpd.read_file("Spatial Files/oakland_bike_collisions.shp")

In [ ]:
# Create smaller simpler dataframe
# Saving Case_ID, Crash Severity, and geometry
# Crash severity is in the column labelled "COLLISIO_1

ped_cr_sh = ped_cr[['CASE_ID','COLLISIO_1','geometry']]
ped_cr_sh = ped_cr_sh.rename(columns={"CASE_ID": "ped_ID", "COLLISIO_1": "ped_sev"})
ped_cr_sh['count_ped'] = 1
bike_cr_sh = bike_cr[['CASE_ID','COLLISIO_1','geometry']]
bike_cr_sh = bike_cr_sh.rename(columns={"CASE_ID": "bike_ID", "COLLISIO_1": "bike_sev"})
bike_cr_sh['count_bike'] = 1

In [ ]:
# Reproject census block group data into the same projection

bl_simp = bl_simp.to_crs(ped_cr_sh.crs)
place_simp = place_simp.to_crs(ped_cr_sh.crs)

In [ ]:
# Remove crashes that are outside of Oakland borders

ped_cr_sh = gpd.sjoin(ped_cr_sh, place_simp, how = 'inner').drop(columns = ['NAME','index_right'])
bike_cr_sh = gpd.sjoin(bike_cr_sh, place_simp, how = 'inner').drop(columns = ['NAME','index_right'])

In [ ]:
# Plot what we have

ax = bl_simp.plot(figsize = (10, 10), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.5)
ax = ped_cr_sh.plot(ax =ax, color = 'Blue', alpha = 0.5) # alpha allows you to make layers transparent
bike_cr_sh.plot(ax =ax, color = 'Purple', alpha = 0.5).set_axis_off()

### 2. Modifying Spatial Data
#### ACTION STEP - 1
#### 2.1 Determine Distance Consideration
How close should a crash be to a block group to be considered an impact on residents?<br/>
We can use buffering to change the area under consideration. 

In [ ]:
# We can look at the CRS to determine the projection being used and the scale for measurements.
# We are using us-ft.

ped_cr.crs

#### ACTION STEP IS HERE
How close to a block group should be a crash be for us to consider it
an impact on them? <br>
You decide? (Remember we are measuring in feet)

In [ ]:
ped_buffer = 5280/4 # Quarter mile
bike_buffer = (5280) # Mile

In [ ]:
# Transform point geometry into polygons
# Geometry become "impact area" of a crash
ped_cr_sh['geometry'] = ped_cr_sh['geometry'].buffer(ped_buffer)
bike_cr_sh['geometry'] = bike_cr_sh['geometry'].buffer(bike_buffer)

In [ ]:
# See what we created
# Circles show the "influence" areas for crashes
# Alpha - use for creating transparency

ax = bl_simp.plot(figsize = (10, 10), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.1)
ax = bike_cr_sh.plot(ax =ax, facecolor = "none", edgecolor = "Purple", alpha = 0.1)
ped_cr_sh.plot(ax =ax, facecolor = "none", edgecolor = "Blue", alpha = 0.1).set_axis_off()

### ACTION STEP - 2
#### 2.2 Weight crashes by severity
We are now going to create a metric based on the severity of the crashes. <br>
How should we compare more or less severe crashes. Not all crashes are equal <br>
but how should we compare them to create a composite score?

There are four types of severities in the dataset (note Property Damage Only crashes are not included).

##### What severity should be given to each crash

In [ ]:
# You decide... the default gives fatal and severe crashes twice as much weight.

fatal = 5
severe = 5
visible = 1
comp_pain = 1

##### You can ignore this part of the code if you would like...
It is creating the weighted score.

In [ ]:
# This part of the code adds a severity weight column

ped_cr_sh['p_sev_wt'] = 1
ped_cr_sh['p_sev_wt'][ped_cr_sh['ped_sev'] == 1] = fatal
ped_cr_sh['p_sev_wt'][ped_cr_sh['ped_sev'] == 2] = severe
ped_cr_sh['p_sev_wt'][ped_cr_sh['ped_sev'] == 3] = visible
ped_cr_sh['p_sev_wt'][ped_cr_sh['ped_sev'] == 4] = comp_pain

bike_cr_sh['b_sev_wt'] = 1
bike_cr_sh['b_sev_wt'][bike_cr_sh['bike_sev'] == 1] = fatal
bike_cr_sh['b_sev_wt'][bike_cr_sh['bike_sev'] == 2] = severe
bike_cr_sh['b_sev_wt'][bike_cr_sh['bike_sev'] == 3] = visible
bike_cr_sh['b_sev_wt'][bike_cr_sh['bike_sev'] == 4] = comp_pain

In [ ]:
# Join crashes to block groups. Sjoin will create a location record for each overlap of a crash buffer and the block group

bl_ped = gpd.sjoin(bl_simp, ped_cr_sh, how = 'left').drop(columns = ['index_right'])
bl_bike = gpd.sjoin(bl_simp, bike_cr_sh, how = 'left').drop(columns = ['index_right'])

In [ ]:
# Summarizes the data into tables organized by block group.

bl_sum_ped = bl_ped.groupby(['GEOID_2'])['count_ped','p_sev_wt'].sum().reset_index()
bl_sum_bike = bl_bike.groupby(['GEOID_2'])['count_bike','b_sev_wt'].sum().reset_index()

In [ ]:
# Joins the data into a single table.

bl_analysis = bl_sum_ped.merge(bl_sum_bike, on = 'GEOID_2')
bl_analysis = bl_simp.merge(bl_analysis, on = 'GEOID_2')

### 3. Fun with Maps!!!
Python maps are not a pretty as what you can make in other programs...
BUT you can make a lot of maps really quickly. The code below creates <br>
a map based on the information in each column in the list.
Feel free to speal the code to generate your own maps.

In [ ]:
# This is a usefule bit of code for clipping your spatial files by a box around your data
# The script creates a box around the data you want to display
# Then you can use the polygon to "clip" down features you want to include in your map.

from shapely.geometry import Point, Polygon
bbox = bl_analysis.total_bounds

p1 = Point(bbox[0], bbox[3])
p2 = Point(bbox[2], bbox[3])
p3 = Point(bbox[2], bbox[1])
p4 = Point(bbox[0], bbox[1])

np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])

bb_polygon = Polygon([np1, np2, np3, np4])
cut_shape = gpd.GeoDataFrame(gpd.GeoSeries(bb_polygon), columns=['geometry'])

In [ ]:
# Added geographic characteristics
# These are so I can make an ok -> good looking visualization
# Make sure to set layer to same projections otherwise the clip doesn't work

# Places
census_places = census_places.to_crs(bl_analysis.crs) # Census Bureau
# Major roads from Census
roads = gpd.read_file("Spatial Files/tl_2019_06_prisecroads.shp") # Census Bureau
roads = roads.to_crs(bl_analysis.crs)
# Parks from Oakland Opend Data Portal
parks = gpd.read_file('Spatial Files/geo_export_2c4cc7cb-4b1e-4799-91d1-693aa89a9e80.shp') # City of Oakland
parks = parks.to_crs(bl_analysis.crs)
# Water
water = gpd.read_file('Spatial Files/water.shp') # MTC ABAG
water = water.to_crs(bl_analysis.crs)
water2 = gpd.read_file('Spatial Files/WaterMerged.shp') # MTC ABAG
water2 = water2.to_crs(bl_analysis.crs)

In [ ]:
# Clip using the overlay command from geopandas

census_p = gpd.overlay(census_places, cut_shape, how = 'intersection')
roads['geometry'] = roads['geometry'].buffer(60)
roads = gpd.overlay(roads, cut_shape, how = 'intersection')
water = gpd.overlay(water, cut_shape,how = 'intersection')
water2 = gpd.overlay(water2, cut_shape, how = 'intersection')

In [ ]:
for x in ['count_ped', 'p_sev_wt','count_bike','p_sev_wt']:
    ax = census_p.plot(figsize = (15, 15), color = 'whitesmoke', edgecolor = "grey", linewidth = 0.5)
    ax = bl_analysis.plot(ax =ax, scheme = "quantiles", k = 6, column = x, cmap = "BuPu",edgecolor = "lightgrey", linewidth = 0.5, legend = True)
    ax = parks.plot(ax = ax, color = 'lightgreen')
    ax = roads.plot(ax = ax, color = 'white')
    ax = census_p.plot(ax =ax, color = 'None', edgecolor = "grey", linewidth = 0.5)
    ax = water.plot(ax = ax, color = 'lightblue')
    ax = water2.plot(ax = ax, color = 'lightblue')
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0., 0., 0.2, 0.2))
    ax.set_axis_off()
    ax.set_title(x, fontsize=15, loc = 'left')

### 4. Managing Data
#### 4.1 Normalizing the data
For our purposes we will want to normalize the variables summarizing the crash data. <br>
This will allow us to more easily factor the results into a composite index in the next step.

In [ ]:
# Create scores normalized as percent of highest value
bl_analysis['c_ped_perc'] = bl_analysis['count_ped']/bl_analysis['count_ped'].max()
bl_analysis['sev_ped_perc'] = bl_analysis['p_sev_wt']/bl_analysis['p_sev_wt'].max()
bl_analysis['c_bike_perc'] = bl_analysis['count_bike']/bl_analysis['count_bike'].max()
bl_analysis['sev_bike_perc'] = bl_analysis['b_sev_wt']/bl_analysis['b_sev_wt'].max()

#### 4.2 Add Census Information
This step brings in census data that we've downloaded from social explorer and done some minor <br>
cleaning. It includes a set of variables described in a table we've hopefully provided to you for <br>
the lab (fingers crossed).

In [ ]:
indices = pd.read_csv("data/alamedacounty_mostly_computed_indices.csv").fillna(0)

In [ ]:
indices.columns

##### Join the table to our work from above based on the block group
We've also included some work to drop columns that are not necessary to keep things more clean.

In [ ]:
bl_analysis['join_id'] = bl_analysis['GEOID_2']

In [ ]:
indices['join_id'] = indices['Geo_GEOID'].str[7:]

In [ ]:
bl_analysis = bl_analysis.merge(indices, on = 'join_id').drop(columns = ['join_id', 'Geo_FIPS', 'Geo_GEOID', 'Geo_NAME', 'Geo_QName',
       'Geo_STUSAB', 'Geo_SUMLEV', 'Geo_GEOCOMP', 'Geo_FILEID', 'Geo_LOGRECNO',
       'Geo_US', 'Geo_REGION', 'Geo_DIVISION', 'Geo_STATECE', 'Geo_STATE',
       'Geo_COUNTY', 'Geo_COUSUB'])

##### Let's look at what we have!
We are going to use that same mapping code from before to see what the variables look like.

In [ ]:
bl_analysis.columns

In [ ]:
for x in ['avg.hh.size', 'coc.population', 'total.population',
       'elderly.population', 'households', 'limited.english.households',
       'poverty.population', 'pop.poverty.determined', 'single.parent.fam',
       'families', 'youth.population', 'occupied.housing.units',
       'zero.vehicle.hh', 'coc.population.pct', 'elderly.population.pct',
       'limited.english.households.pct', 'poverty.population.pct',
       'single.parent.fam.pct', 'youth.population.pct', 'zero.vehicle.hh.pct',
       'coc.pop.norm', 'elderly.pop.norm', 'limited.english.households.norm',
       'poverty.population.norm', 'single.parent.fam.norm',
       'youth.population.norm', 'zero.vehicle.hh.norm']:
    ax = census_p.plot(figsize = (15, 15), color = 'whitesmoke', edgecolor = "grey", linewidth = 0.5)
    ax = bl_analysis.plot(ax =ax, scheme = "quantiles", k = 6, column = x, cmap = "BuPu",edgecolor = "lightgrey", linewidth = 0.5, legend = True)
    ax = parks.plot(ax = ax, color = 'lightgreen')
    ax = roads.plot(ax = ax, color = 'white')
    ax = census_p.plot(ax =ax, color = 'None', edgecolor = "grey", linewidth = 0.5)
    ax = water.plot(ax = ax, color = 'lightblue')
    ax = water2.plot(ax = ax, color = 'lightblue')
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0., 0., 0.2, 0.2))
    ax.set_axis_off()
    ax.set_title(x, fontsize=15, loc = 'left')

### 4.4 Create an Index
#### ACTION STEP - 3
#### Where should we prioritize active transportation investments?
We have all the data. But now the hard questions... <br>
What factors should we consider to prioritize resources. <br>
We included three starter indecise: (1) Crash Only, (2) Census Factors Only, (3) Simple All Factors.
They are not that good. Make us something better... whatever that means. <br>
Go down to indecis 4 and 5 to start making your own.

In [ ]:
bl_analysis['index1'] = \
        bl_analysis['c_ped_perc'] * 1 + \
        bl_analysis['sev_ped_perc'] * 1 + \
        bl_analysis['c_bike_perc'] * 1 + \
        bl_analysis['sev_bike_perc'] * 1

In [ ]:
bl_analysis['index2'] = \
        bl_analysis['coc.population.pct'] * 1 + \
        bl_analysis['elderly.population.pct'] * 1 + \
        bl_analysis['limited.english.households.pct'] * 1 + \
        bl_analysis['poverty.population.pct'] * 1 + \
        bl_analysis['single.parent.fam.pct'] * 1 + \
        bl_analysis['youth.population.pct'] * 1 + \
        bl_analysis['zero.vehicle.hh.pct'] * 1 + \
        bl_analysis['coc.pop.norm'] * 1 + \
        bl_analysis['elderly.pop.norm'] * 1 + \
        bl_analysis['limited.english.households.norm'] * 1 + \
        bl_analysis['poverty.population.norm'] * 1 + \
        bl_analysis['single.parent.fam.norm'] * 1 + \
        bl_analysis['youth.population.norm'] * 1 + \
        bl_analysis['zero.vehicle.hh.norm'] * 1

In [ ]:
bl_analysis['index3'] = \
        bl_analysis['coc.population.pct'] * 1 + \
        bl_analysis['elderly.population.pct'] * 1  + \
        bl_analysis['limited.english.households.pct'] * 1 + \
        bl_analysis['poverty.population.pct'] * 1 + \
        bl_analysis['single.parent.fam.pct'] * 1 + \
        bl_analysis['youth.population.pct'] * 1 + \
        bl_analysis['zero.vehicle.hh.pct'] * 1 + \
        bl_analysis['coc.pop.norm'] * 1  + \
        bl_analysis['elderly.pop.norm'] * 1 + \
        bl_analysis['limited.english.households.norm'] * 1 + \
        bl_analysis['poverty.population.norm'] * 1 + \
        bl_analysis['single.parent.fam.norm'] * 1 + \
        bl_analysis['youth.population.norm'] * 1 + \
        bl_analysis['zero.vehicle.hh.norm'] * 4 + \
        bl_analysis['count_ped']/bl_analysis['count_ped'].max() * 2 + \
        bl_analysis['p_sev_wt']/bl_analysis['p_sev_wt'].max() * 2 + \
        bl_analysis['count_bike']/bl_analysis['count_bike'].max() * 2 + \
        bl_analysis['b_sev_wt']/bl_analysis['b_sev_wt'].max() * 2

##### The easiest way to make your index is to change the weighting factor already in the code
0 is the same as deleting the factor
And... Keep creating if we have actually left enough time for you.

In [ ]:
bl_analysis['index4'] = \
        bl_analysis['coc.population.pct'] * 5 + \
        bl_analysis['elderly.population.pct'] * 2  + \
        bl_analysis['limited.english.households.pct'] * 0 + \
        bl_analysis['poverty.population.pct'] * 5 + \
        bl_analysis['single.parent.fam.pct'] * 0 + \
        bl_analysis['youth.population.pct'] * 8 + \
        bl_analysis['zero.vehicle.hh.pct'] * 8 + \
        bl_analysis['coc.pop.norm'] * 10  + \
        bl_analysis['elderly.pop.norm'] * 0 + \
        bl_analysis['limited.english.households.norm'] * 0 + \
        bl_analysis['poverty.population.norm'] * 0 + \
        bl_analysis['single.parent.fam.norm'] * 0 + \
        bl_analysis['youth.population.norm'] * 0 + \
        bl_analysis['zero.vehicle.hh.norm'] * 0 + \
        bl_analysis['count_ped']/bl_analysis['count_ped'].max() * 0 + \
        bl_analysis['p_sev_wt']/bl_analysis['p_sev_wt'].max() * 10 + \
        bl_analysis['count_bike']/bl_analysis['count_bike'].max() * 0 + \
        bl_analysis['b_sev_wt']/bl_analysis['b_sev_wt'].max() * 10

In [ ]:
bl_analysis['index5'] = \
        bl_analysis['coc.population.pct'] * 1 + \
        bl_analysis['elderly.population.pct'] * 1  + \
        bl_analysis['limited.english.households.pct'] * 1 + \
        bl_analysis['poverty.population.pct'] * 1 + \
        bl_analysis['single.parent.fam.pct'] * 1 + \
        bl_analysis['youth.population.pct'] * 1 + \
        bl_analysis['zero.vehicle.hh.pct'] * 1 + \
        bl_analysis['coc.pop.norm'] * 1  + \
        bl_analysis['elderly.pop.norm'] * 1 + \
        bl_analysis['limited.english.households.norm'] * 1 + \
        bl_analysis['poverty.population.norm'] * 1 + \
        bl_analysis['single.parent.fam.norm'] * 1 + \
        bl_analysis['youth.population.norm'] * 1 + \
        bl_analysis['zero.vehicle.hh.norm'] * 4 + \
        bl_analysis['count_ped']/bl_analysis['count_ped'].max() * 2 + \
        bl_analysis['p_sev_wt']/bl_analysis['p_sev_wt'].max() * 2 + \
        bl_analysis['count_bike']/bl_analysis['count_bike'].max() * 2 + \
        bl_analysis['b_sev_wt']/bl_analysis['b_sev_wt'].max() * 2

In [ ]:
for x in ['index1','index2','index3','index4','index5']:
    ax = census_p.plot(figsize = (15, 15), color = 'whitesmoke', edgecolor = "grey", linewidth = 0.5)
    ax = bl_analysis.plot(ax =ax, scheme = "quantiles", k = 6, column = x, cmap = "pink_r",edgecolor = "lightgrey", linewidth = 0.5, legend = True)
    ax = parks.plot(ax = ax, color = 'lightgreen')
    ax = roads.plot(ax = ax, color = 'white')
    ax = census_p.plot(ax =ax, color = 'None', edgecolor = "grey", linewidth = 0.5)
    ax = water.plot(ax = ax, color = 'lightblue')
    ax = water2.plot(ax = ax, color = 'lightblue')
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0., 0., 0.2, 0.2))
    ax.set_axis_off()
    ax.set_title(x, fontsize=15, loc = 'left')

### Final Action Item
#### Pick your preferred index and send it to us.
At the end we will put them on the board. <br>
For simplicity just copy the image and email it to us.
